# Examen Bimestral – Diseño de un Sistema Básico de Recuperación de Información

## Instrucciones:
En este examen, los estudiantes deberán diseñar e implementar un sistema básico de recuperación de
información utilizando la base de datos **Rotten Tomatoes movies and critic reviews** disponible en
Kaggle. El objetivo es responder consultas relacionadas con la temática de las películas y sus
características.

## 1. Fases de la implementación

### 1.1 Adquisición de datos


* Descargar el Corpus `rotten_tomatoes_movies` y `rotten_tomatoes_critic_reviews` que se encuentran en Kaggle
* Descomprimir y colocamos nuestros archivos en la carpeta de **Archivos** que nos ofrece Google colab.

![Descripción de la imagen](../images/addcorpus.png)

In [33]:
import pandas as pd # manipular datos en formate DataFrames
import numpy as np # Operaciones numéricas
import re # Uso en expresiones regulares para la limpieza de datos
from sklearn.feature_extraction.text import TfidfVectorizer # Conversión a vector TF-IDF
from sklearn.metrics.pairwise import cosine_similarity # Calculo para la similitud coseno
from nltk.corpus import stopwords  # Acceder a una lista de palabras comunes (stopwords) en varios idiomas.
from nltk.stem.porter import PorterStemmer # Para realizar stemming (reducir palabras a su raíz).
from nltk.tokenize import word_tokenize # División de texto en tokens (palabras)
from collections import defaultdict
import random
import nltk

In [2]:
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Declaraciones previas
# Crear una lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

# Crear un objeto PorterStemmer
stemmer = PorterStemmer()

In [4]:
# Cargar datasets
movies_df = pd.read_csv('../data/rotten_tomatoes_movies.csv')
reviews_df = pd.read_csv('../data/rotten_tomatoes_critic_reviews.csv')

In [5]:
print("Contenido de movies_df:")
display(movies_df.head())

Contenido de movies_df:


,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [6]:
print("Columnas de movies_df:")
print(movies_df.columns)

Columnas de movies_df:
Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')


In [7]:
print("Contenido de reviews_df:")
display(reviews_df.head())

Contenido de reviews_df:


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [8]:
print("Columnas de reviews_df:")
print(reviews_df.columns)

Columnas de reviews_df:
Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content'],
      dtype='object')


### 1.2. Preprocesamiento

La función ***preprocess_text*** toma datos de texto sin procesar, los limpia eliminando caracteres y palabras innecesarias, reduce las palabras a su raíz y devuelve una versión procesada del texto que es más adecuada para el análisis

#### 1.2.1. Conversión a minúsculas

In [9]:
# Visualizar el primer dato original
print("Texto original:")
print(reviews_df['review_content'].iloc[0])

# Convertir a minúsculas
reviews_df['review_content'] = reviews_df['review_content'].str.lower()

# Visualizar el resultado del primer dato después de convertir a minúsculas
print("\nTexto en minúsculas:")
print(reviews_df['review_content'].iloc[0])

Texto original:
A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle

Texto en minúsculas:
a fantasy adventure that fuses greek mythology to contemporary american places and values. anyone around 15 (give or take a couple of years) will thrill to the visual spectacle


#### 1.2.2. Eliminación de caracteres especiales

In [10]:
# Asegurarse de que todos los valores sean cadenas, reemplazando NaN por una cadena vacía
reviews_df['review_content'] = reviews_df['review_content'].fillna('').astype(str)

# Eliminar caracteres especiales usando expresiones regulares
reviews_df['review_content'] = reviews_df['review_content'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Visualizar el resultado del primer dato después de eliminar caracteres especiales
print("\nTexto sin caracteres especiales:")
print(reviews_df['review_content'].iloc[0])


Texto sin caracteres especiales:
a fantasy adventure that fuses greek mythology to contemporary american places and values anyone around  give or take a couple of years will thrill to the visual spectacle


#### 1.2.3. Eliminación de stopwords

In [11]:
# Eliminar las stopwords del texto
reviews_df['review_content'] = reviews_df['review_content'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)

# Visualizar el resultado del primer dato después de eliminar stopwords
print("\nTexto sin stopwords:")
print(reviews_df['review_content'].iloc[0])


Texto sin stopwords:
fantasy adventure fuses greek mythology contemporary american places values anyone around give take couple years thrill visual spectacle


#### 1.2.4. Tokenización

In [12]:
# Función para tokenizar texto
def tokenizar_texto(texto):
    # Utilizamos word_tokenize de NLTK para dividir en tokens
    tokens = word_tokenize(texto)
    return tokens

In [13]:
# Aplicar la función a la columna 'review_content'
reviews_df['review_tokens'] = reviews_df['review_content'].apply(tokenizar_texto)

# Visualizar el resultado del primer dato después de la tokenización
print("\nTexto tokenizado:")
print(reviews_df['review_tokens'].iloc[0])


Texto tokenizado:
['fantasy', 'adventure', 'fuses', 'greek', 'mythology', 'contemporary', 'american', 'places', 'values', 'anyone', 'around', 'give', 'take', 'couple', 'years', 'thrill', 'visual', 'spectacle']


#### 1.2.5. Stemming

In [14]:
# Aplicar stemming a cada token del texto
reviews_df['review_stemmed'] = reviews_df['review_tokens'].apply(
    lambda tokens: [stemmer.stem(token) for token in tokens]
)

# Visualizar el resultado del primer dato después de aplicar stemming
print("\nTexto después del stemming:")
print(reviews_df['review_stemmed'].iloc[0])


Texto después del stemming:
['fantasi', 'adventur', 'fuse', 'greek', 'mytholog', 'contemporari', 'american', 'place', 'valu', 'anyon', 'around', 'give', 'take', 'coupl', 'year', 'thrill', 'visual', 'spectacl']


#### 1.2.6. Normalización

In [15]:
# Unificar las palabras preprocesadas en una sola cadena para cada texto
reviews_df['review_normalized'] = reviews_df['review_stemmed'].apply(lambda tokens: ' '.join(tokens))

# Visualizar el resultado del primer dato después de la normalización
print("\nTexto normalizado:")
print(reviews_df['review_normalized'].iloc[0])


Texto normalizado:
fantasi adventur fuse greek mytholog contemporari american place valu anyon around give take coupl year thrill visual spectacl


#### 1.2.7. Guardado (Preprocesamiento)

Generamo un archivo CSV que contiene a la varible de `review_content` luego de todo el proceso del preprocesamiento y listo para utilizarlo.

In [16]:
reviews_df.to_csv('../data/reviews_preprocessed.csv', index=False)

![Descripción de la imagen](../images/corpusProcesado.png)

### 1.3.  Construcción del Sistema

#### 1.3.1. Cargar (Preprocesamiento)

In [17]:
# Cargar datasets
reviewsp_df = pd.read_csv('../data/reviews_preprocessed.csv')

In [18]:
print("Contenido de reviewsp__df:")
display(reviewsp_df.head())

Contenido de reviewsp__df:


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,review_tokens,review_stemmed,review_normalized
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,fantasy adventure fuses greek mythology contem...,"['fantasy', 'adventure', 'fuses', 'greek', 'my...","['fantasi', 'adventur', 'fuse', 'greek', 'myth...",fantasi adventur fuse greek mytholog contempor...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,uma thurman medusa gorgon coiffure writhing sn...,"['uma', 'thurman', 'medusa', 'gorgon', 'coiffu...","['uma', 'thurman', 'medusa', 'gorgon', 'coiffu...",uma thurman medusa gorgon coiffur writh snake ...
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,topnotch cast dazzling special effects tide te...,"['topnotch', 'cast', 'dazzling', 'special', 'e...","['topnotch', 'cast', 'dazzl', 'special', 'effe...",topnotch cast dazzl special effect tide teen n...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,whether audiences get behind lightning thief h...,"['whether', 'audiences', 'get', 'behind', 'lig...","['whether', 'audienc', 'get', 'behind', 'light...",whether audienc get behind lightn thief hard p...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,whats really lacking lightning thief genuine s...,"['whats', 'really', 'lacking', 'lightning', 't...","['what', 'realli', 'lack', 'lightn', 'thief', ...",what realli lack lightn thief genuin sens wond...


In [27]:
print("Columnas de reviewsp_df:")
print(reviewsp_df.columns)

Columnas de reviewsp_df:
Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'review_tokens', 'review_stemmed', 'review_normalized'],
      dtype='object')


#### 1.3.2. Vectorización (TF-IDF)

In [28]:
# Reemplazar valores NaN en la columna 'review_normalized' con cadenas vacías
reviewsp_df['review_normalized'] = reviewsp_df['review_normalized'].fillna('')

# Instanciar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Limitar a las 10,000 palabras más frecuentes

# Aplicar la vectorización a la columna 'review_normalized'
tfidf_matrix = tfidf_vectorizer.fit_transform(reviewsp_df['review_normalized'])

# Mostrar solo las primeras 20 palabras (usando la matriz dispersa)
print("Primeras 20 palabras:")
print(tfidf_vectorizer.get_feature_names_out()[:20])

# Convertir una muestra dispersa a un DataFrame
sample_tfidf_df = pd.DataFrame(tfidf_matrix[:10].toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print(sample_tfidf_df)

Primeras 20 palabras:
['aardman' 'aaron' 'ab' 'abandon' 'abba' 'abbey' 'abbi' 'abbott' 'abduct'
 'abel' 'abid' 'abil' 'abject' 'abl' 'abli' 'aboard' 'abomin' 'abort'
 'abound' 'aboveaverag']
   aardman  aaron   ab  abandon  abba  abbey  abbi  abbott  abduct  abel  ...  \
0      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
1      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
2      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
3      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
4      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
5      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
6      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
7      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   
8      0.0    0.0  0.0      0.0   0.0    0.0   0.0     0.0     0.0   0.0  ...   

#### 1.3.3. Indexación

In [20]:
def construir_indice_invertido(dataframe, vectorizador, matriz_tfidf):
    indice_invertido = defaultdict(list)  # Diccionario para el índice invertido
    vocabulario = vectorizador.get_feature_names_out()  # Obtener palabras del vocabulario
    
    # Iterar sobre los valores no nulos en la matriz dispersa
    coo_matrix = matriz_tfidf.tocoo()  # Convertir a formato COO (Coordinate Format) para iteración eficiente
    for rotten_tomatoes_idx, term_idx in zip(coo_matrix.row, coo_matrix.col):
        rotten_tomatoes_id = dataframe['rotten_tomatoes_link'].iloc[rotten_tomatoes_idx]  # Obtener rotten_tomatoe_link de la review
        termino = vocabulario[term_idx]  # Obtener el término del vocabulario
        indice_invertido[termino].append(rotten_tomatoes_id)  # Guardar solo el rotten_tomatoe_link
    
    return indice_invertido

In [21]:
# Construir el índice invertido sin pesos
indice_invertido_tfidf = construir_indice_invertido(
    dataframe=reviewsp_df,
    vectorizador=tfidf_vectorizer,
    matriz_tfidf=tfidf_matrix
)

# Mostrar los primeros términos del índice invertido
for i, (termino, entradas) in enumerate(indice_invertido_tfidf.items()):
    if i >= 3:  # Mostrar solo los primeros 3 términos
        break
    print(f"Término: '{termino}' -> Entradas: {entradas}")

Término: 'spectacl' -> Entradas: ['m/0814255', 'm/0814255', 'm/0814255', 'm/10000_bc', 'm/10000_bc', 'm/10000_bc', 'm/10000_bc', 'm/10000_bc', 'm/10000_bc', 'm/1000355-adventures_of_robin_hood', 'm/10004504-ultraviolet', 'm/1000617-aliens', 'm/1000626-all_about_eve', 'm/10007985-happening', 'm/10008502-christmas_carol', 'm/10008502-christmas_carol', 'm/10008705-mr_untouchable', 'm/10009001-the_freshman', 'm/10009083-land_of_the_lost', 'm/10009526-public_enemies', 'm/10009526-public_enemies', 'm/10009598-surrogates', 'm/10010066-push', 'm/10011457-ong_bak_2_2009', 'm/10011582-TRON_legacy', 'm/10011582-TRON_legacy', 'm/10011582-TRON_legacy', 'm/1001193-around_the_world_in_80_days', 'm/1001193-around_the_world_in_80_days', 'm/10012058-prodigal_sons', 'm/1001902-beauty_and_the_beast', 'm/1002448-birds', 'm/1003499-captain_blood', 'm/1004330-cleopatra', 'm/1004330-cleopatra', 'm/1004330-cleopatra', 'm/1005152-damn_yankees', 'm/1006042-dodge_city', 'm/1006364-dune', 'm/1007538-flesh_and_bloo

#### 1.3.4. Diseño del Motor de Búsqueda

##### 1.3.4.1. Desarrollar la lógica para procesar consultas de usuarios

In [22]:
# Función para limpiar una consulta
def limpiar_normalizar_consulta(consulta):
    # Convertir a minúsculas
    consulta = consulta.lower()
    # Eliminar caracteres especiales
    consulta = re.sub(r'[^a-zA-Z\s]', '', consulta)
    # Tokenizar la consulta
    tokens = word_tokenize(consulta)
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Aplicar stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

In [23]:
# Ejemplo de limpieza
query = "amazing"
limpiar_normalizar_consulta(query)

['amaz']

In [24]:
# Función para procesar una consulta y buscar documentos relevantes
def procesar_consulta(consulta, indice_invertido):
    # Limpieza y normalización de la consulta
    consulta_normalizada = limpiar_normalizar_consulta(consulta)
    # Inicializar un conjunto para almacenar documentos relevantes
    documentos_relevantes = set()
    # Buscar cada término en el índice invertido
    for termino in consulta_normalizada:
        if termino in indice_invertido:
            # Agregar los documentos relevantes asociados al término
            documentos_relevantes.update([doc[0] for doc in indice_invertido[termino]])  # Solo el ID del documento
    return documentos_relevantes

##### 1.3.4.2. Uso de algoritmo de similitud coseno y Ranking

In [38]:
# Función para procesar la consulta
def procesar_consulta(consulta, vectorizador, matriz_tfidf, dataframe, top_n=10):
    # Limpiar y normalizar la consulta
    consulta_normalizada = ' '.join(limpiar_normalizar_consulta(consulta))
    
    # Convertir la consulta en un vector TF-IDF
    consulta_tfidf = vectorizador.transform([consulta_normalizada])
    
    # Calcular similitudes de coseno entre la consulta y todos los documentos
    similitudes = cosine_similarity(consulta_tfidf, matriz_tfidf)
    
    # Obtener índices de documentos ordenados por similitud descendente
    reviews_ordenados_idx = np.argsort(-similitudes[0])
    
    # Recuperar los primeros 'top_n' IDs de los documentos relevantes
    reviews_relevantes = [
        (dataframe.iloc[idx]['rotten_tomatoes_link'], similitudes[0, idx]) 
        for idx in reviews_ordenados_idx[:top_n]  # Limitar a los primeros 'top_n' resultados
        if similitudes[0, idx] > 0
    ]
    
    return reviews_relevantes

##### 1.3.5. Ejemplo de Busqueda usando la consulta "space travel"

In [51]:
# Ejemplo de uso
consulta_usuario = "space travel"
reviews_relevantes = procesar_consulta(
    consulta_usuario, tfidf_vectorizer, tfidf_matrix, reviewsp_df, top_n=10
)

# Mostrar resultados
if reviews_relevantes:
    print("Documentos relevantes encontrados (ordenados por similitud):")
    for reviews_id, similitud in reviews_relevantes:
        # Mostrar texto limpio asociado a cada documento relevante
        print(f"Reviews ID: {reviews_id} | Similitud Coseno: {similitud:.4f}")
else:
    print("No se encontraron documentos relevantes para la consulta.")

Documentos relevantes encontrados (ordenados por similitud):
Reviews ID: m/iron_sky | Similitud Coseno: 0.7661
Reviews ID: m/first_man | Similitud Coseno: 0.7136
Reviews ID: m/ad_astra | Similitud Coseno: 0.7025
Reviews ID: m/a_wrinkle_in_time_2018 | Similitud Coseno: 0.6667
Reviews ID: m/thank_you_for_your_service_2017 | Similitud Coseno: 0.6612
Reviews ID: m/the_trip_to_spain_2017 | Similitud Coseno: 0.6206
Reviews ID: m/apollo_11_2019 | Similitud Coseno: 0.5764
Reviews ID: m/it_the_terror_from_beyond_space | Similitud Coseno: 0.5359
Reviews ID: m/interstellar_2014 | Similitud Coseno: 0.5256
Reviews ID: m/captain_abu_raed | Similitud Coseno: 0.5243


#### 1.3.6. Uso de "rotten_tomatoes_movies" para obtener el nombre de la película

In [52]:
# Mostrar resultados con el título de la película
if reviews_relevantes:
    print("Documentos relevantes encontrados (ordenados por similitud):")
    for reviews_id, similitud in reviews_relevantes:
        # Buscar el título de la película en movies_df usando el rotten_tomatoes_link
        movie_title = movies_df.loc[movies_df['rotten_tomatoes_link'] == reviews_id, 'movie_title'].values
        if movie_title:
            print(f"Película: {movie_title[0]}")
        else:
            print(f"ID: {reviews_id} no tiene título asociado")
else:
    print("No se encontraron documentos relevantes para la consulta.")

Documentos relevantes encontrados (ordenados por similitud):
Película: Iron Sky
Película: First Man
Película: Ad Astra
Película: A Wrinkle in Time
Película: Thank You for Your Service
Película: The Trip to Spain
Película: Apollo 11
Película: It! The Terror From Beyond Space
Película: Interstellar
Película: Captain Abu Raed


### 1.4. Análisis de Resultados

Partiendo de lo siguiente, de los resultados obtenidos por mi SRI (Sistema de Recuperación) las películas que tienen relación con la temática de "space travel" (viajes espaciales) de la lista proporcionada son:

- Iron Sky
- First Man
- Ad Astra
- Apollo 11
- Interstellar
- It! The Terror From Beyond Space

Estas películas están directamente relacionadas con el tema de los viajes espaciales, ya sea a través de misiones espaciales, exploración del espacio o experiencias en el espacio. Por otra parte, Las demás películas de la lista tienen temáticas que no están directamente relacionadas con los viajes espaciales, pero aún pueden tratar de otros aspectos como la historia humana, la ciencia ficción o el drama, aunque no se centran en el espacio.


Por tanto, puedo colegir que mi Sistema de Recuperación de Información ha proporcionado resultados relacionados con la temática de "space travel", Sin embargo, no todas las películas de la lista están directamente relacionadas con los viajes espaciales, lo que sugiere que el sistema aún puede mejorar en la precisión de sus resultados, ya que algunas películas no se ajustan completamente al tema de la consulta.